<a href="https://colab.research.google.com/github/elenancalima/Troph_Min_5to2/blob/main/Simple_troph_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Purpose:
# - Ensure required Python packages (versions) are present.
# - If anything is installed/updated, automatically restart the runtime
#   so subsequent cells see the fresh libs.

import sys, subprocess

def _pip_install(spec: str):
    print(f"[env] pip install -U {spec}")
    subprocess.check_call([sys.executable, "-m", "pip", "install", "-Uq", spec])

def _pkg_version(mod_name: str):
    try:
        import importlib
        m = importlib.import_module(mod_name)
        return getattr(m, "__version__", "unknown")
    except Exception:
        return None

def _needs(spec_pkg: str, import_name: str, min_version: str | None) -> bool:
    from packaging.version import Version, InvalidVersion
    v = _pkg_version(import_name)
    if v is None:
        return True
    if not min_version:
        return False
    try:
        return Version(v) < Version(min_version.lstrip(">="))
    except InvalidVersion:
        return True

# ---- requirements (adjust as needed) ----
REQS = [
    # (pip package, python import name, minimum version)
    ("opencv-python-headless", "cv2",   ">=4.5.0"),
    ("imageio",               "imageio",">=2.28.0"),
    ("numpy",                 "numpy",  ">=1.21.0"),
]

updated = False

# Ensure 'packaging' exists for version comparisons
try:
    import packaging  # noqa
except Exception:
    _pip_install("packaging")
    updated = True

for pkg, imp, minv in REQS:
    if _needs(pkg, imp, minv):
        _pip_install(pkg + (f"{minv}" if minv else ""))
        updated = True

# Optional: useful system tool (tar is already present in Colab)
# If you want rsync in the future, uncomment:
# subprocess.run(["apt-get", "update", "-qq"], check=False)
# subprocess.run(["apt-get", "install", "-y", "rsync"], check=False)

# Auto-restart if anything changed
if updated:
    print("\n[env] Runtime will restart to load updated packages. Re-run from Cell 0B afterwards.")
    import IPython
    IPython.get_ipython().kernel.do_shutdown(True)
else:
    # Quick version summary
    import cv2, imageio, numpy as np
    print(f"[env] OK  | cv2={cv2.__version__} | imageio={imageio.__version__} | numpy={np.__version__}")


[env] OK  | cv2=4.12.0 | imageio=2.37.0 | numpy=2.0.2


In [2]:
# === Cell 0B — Drive mount + project paths & globals ===
# Purpose:
# - Mount Google Drive
# - Define canonical paths used across the notebook
# - Prepare local working dirs and Python import path

import os, sys

# 1) Mount Google Drive
try:
    from google.colab import drive  # type: ignore
    drive.mount("/content/drive", force_remount=False)
except Exception as e:
    print("[env] Not in Colab or Drive mount failed:", e)

# 2) Canonical locations
DRIVE_ROOT   = "/content/drive/MyDrive"
LOCAL_STAGE  = "/content/_localstage"   # where we stage video roots locally
NBMODS_DIR   = "/content/_nbmods"       # where the notebook writes its 'modules'
ARCHIVE_NAME = "__inputs_archive__.tar" # canonical inputs archive name
INPUT_DIRS   = ("input_vid", "abdomen_mask", "front_body_mask")

# 3) (Edit these paths to your project)
#    - TEST_VIDEO_ROOT: a single video root to test
#    - BATCH_ROOT: a directory containing many video roots to process
TEST_VIDEO_ROOT = f"{DRIVE_ROOT}/MainConnection_VidRoots/tmpvidroot3"  # <- edit if needed
BATCH_ROOT      = f"{DRIVE_ROOT}/MainConnection_VidRoots"              # <- edit if needed

# 4) Ensure local working dirs exist
os.makedirs(LOCAL_STAGE, exist_ok=True)
os.makedirs(NBMODS_DIR,  exist_ok=True)

# 5) Python import path for notebook-written modules
if NBMODS_DIR not in sys.path:
    sys.path.insert(0, NBMODS_DIR)
if "/content" not in sys.path:
    sys.path.insert(0, "/content")

# 6) Sanity printout
print("[paths]")
print("  DRIVE_ROOT      :", DRIVE_ROOT)
print("  TEST_VIDEO_ROOT :", TEST_VIDEO_ROOT)
print("  BATCH_ROOT      :", BATCH_ROOT)
print("  LOCAL_STAGE     :", LOCAL_STAGE)
print("  NBMODS_DIR      :", NBMODS_DIR)
print("  INPUT_DIRS      :", INPUT_DIRS)
print("  ARCHIVE_NAME    :", ARCHIVE_NAME)

# Optional quick checks (will only warn)
for d in (TEST_VIDEO_ROOT, BATCH_ROOT):
    if not os.path.exists(d):
        print(f"[warn] Path not found yet: {d}")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
[paths]
  DRIVE_ROOT      : /content/drive/MyDrive
  TEST_VIDEO_ROOT : /content/drive/MyDrive/MainConnection_VidRoots/tmpvidroot3
  BATCH_ROOT      : /content/drive/MyDrive/MainConnection_VidRoots
  LOCAL_STAGE     : /content/_localstage
  NBMODS_DIR      : /content/_nbmods
  INPUT_DIRS      : ('input_vid', 'abdomen_mask', 'front_body_mask')
  ARCHIVE_NAME    : __inputs_archive__.tar


In [3]:
# === Cell 1 — Config (control panel) ===
# Use this single cell to control what runs and with which parameters.

#@title Run Controls & Field Params
from dataclasses import dataclass

# ---- Toggles ----
RUN_FAKE  = False  #@param {type:"boolean"}
RUN_TEST  = False   #@param {type:"boolean"}
RUN_BATCH = True  #@param {type:"boolean"}

DEBUG        = True  #@param {type:"boolean"}
DEBUG_FRAME  = 3     #@param {type:"number"}

COPY_RAW_OUTPUTS = False   # default: only upload <stub>_simp.tar
# set True if you also want raw output folders copied to the Drive stub folder


# ---- Batch list file (txt; one subfolder per line under BATCH_ROOT) ----
BATCH_DIR_LIST = f"{DRIVE_ROOT}/MainConnection_VidRoots/batch_dir_list.txt"  #@param {type:"string"}

# ---- Field-data detector PARAMS ----
FIELD_PARAMS = dict(
    scale_factor=5,            # 850 -> 170
    min_duration=4,
    iou_stationary_tol=0.90,   # mean IoU >= 1 - tol
    brightnessThreshold=20,    # R channel threshold (0-255)
    brightPropDelta=0.00,      # abs diff of smoothed red>thr fraction
    smoothingWindow=3,
    coneAngle_deg=35,
    coneLength=80,
    rays_per_cone=9,
    cone_bg_weight=0.05,       # background weighting for cone score
    abdomenToHeadLength=60,
    headRadius=35,
    min_overlap_pairs=2,
    iou_match_threshold=0.01,
)

@dataclass
class CFG:
    # what to run
    run_fake: bool
    run_test: bool
    run_batch: bool
    # debug
    debug: bool
    debug_frame: int
    # paths (from Cell 0B)
    test_video_root: str
    batch_root: str
    batch_dir_list: str
    # params
    params: dict

CFG = CFG(
    run_fake=RUN_FAKE,
    run_test=RUN_TEST,
    run_batch=RUN_BATCH,
    debug=DEBUG,
    debug_frame=int(DEBUG_FRAME),
    test_video_root=TEST_VIDEO_ROOT,  # from Cell 0B
    batch_root=BATCH_ROOT,            # from Cell 0B
    batch_dir_list=BATCH_DIR_LIST,
    params=FIELD_PARAMS,
)

# Short summary
print("[config]")
print("  run_fake        :", CFG.run_fake)
print("  run_test        :", CFG.run_test)
print("  run_batch       :", CFG.run_batch)
print("  debug           :", CFG.debug, " (frame:", CFG.debug_frame, ")")
print("  TEST_VIDEO_ROOT :", CFG.test_video_root)
print("  BATCH_ROOT      :", CFG.batch_root)
print("  BATCH_DIR_LIST  :", CFG.batch_dir_list)
print("  params keys     :", sorted(CFG.params.keys()))

# Optional: gentle warning if batch list file missing
import os
if CFG.run_batch and not os.path.exists(CFG.batch_dir_list):
    print(f"[warn] BATCH_DIR_LIST not found: {CFG.batch_dir_list}")


[config]
  run_fake        : False
  run_test        : False
  run_batch       : True
  debug           : True  (frame: 3 )
  TEST_VIDEO_ROOT : /content/drive/MyDrive/MainConnection_VidRoots/tmpvidroot3
  BATCH_ROOT      : /content/drive/MyDrive/MainConnection_VidRoots
  BATCH_DIR_LIST  : /content/drive/MyDrive/MainConnection_VidRoots/batch_dir_list.txt
  params keys     : ['abdomenToHeadLength', 'brightPropDelta', 'brightnessThreshold', 'coneAngle_deg', 'coneLength', 'cone_bg_weight', 'headRadius', 'iou_match_threshold', 'iou_stationary_tol', 'min_duration', 'min_overlap_pairs', 'rays_per_cone', 'scale_factor', 'smoothingWindow']


In [4]:
# Sections:
#   A) Imports & small utils
#   B) IO helpers
#   C) Geometry / CC helpers
#   D) Visualization helpers
#   E) Listing & alignment
#   F) Cone brightness
#   G) Detector main: process_video_root(...)

# ---------- A) Imports & small utils ----------
import os, re, math, json
import numpy as np
import cv2
from imageio import v3 as iio

def ensure_dirs(*ps):
    for p in ps: os.makedirs(p, exist_ok=True)

# ---------- B) IO helpers ----------
def read_rgb(p):
    im = iio.imread(p)
    if im.ndim != 3 or im.shape[2] != 3:
        raise ValueError(f"expect RGB: {p}")
    return im

def to_bgr(img_rgb: np.ndarray) -> np.ndarray:
    return np.ascontiguousarray(cv2.cvtColor(img_rgb, cv2.COLOR_RGB2BGR))

def read_gray(p):
    im = iio.imread(p)
    if im.ndim == 2:
        g = im
    elif im.ndim == 3:
        g = cv2.cvtColor(to_bgr(im), cv2.COLOR_BGR2GRAY)
    else:
        raise ValueError(f"expect gray or RGB: {p}")
    if g.dtype != np.uint8:
        g = np.clip(g, 0, 255).astype(np.uint8)
    return g

def read_bin(p):
    im = iio.imread(p)
    if im.ndim == 3: im = im[...,0]
    return (im > 127).astype(np.uint8)

# ---------- C) Geometry / CC helpers ----------
def iou(a, b):
    inter = np.bitwise_and(a, b).sum()
    if inter == 0: return 0.0
    union = np.bitwise_or(a, b).sum()
    return float(inter) / float(union)

def clamp(y, x, H, W):
    return max(0, min(H-1, int(round(y)))), max(0, min(W-1, int(round(x))))

def find_nearest_boundary(mask01, cy, cx):
    cnts, _ = cv2.findContours((mask01*255).astype(np.uint8), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
    if not cnts: return cy, cx
    best_d2, by, bx = 1e18, cy, cx
    for c in cnts:
        pts = c.reshape(-1,2)
        dy = pts[:,1]-cy; dx = pts[:,0]-cx
        d2 = dx*dx + dy*dy
        j = int(np.argmin(d2))
        if d2[j] < best_d2:
            best_d2 = float(d2[j]); by, bx = int(pts[j,1]), int(pts[j,0])
    return by, bx

def region_axis_unit(mask01):
    m = cv2.moments((mask01*255).astype(np.uint8), binaryImage=True)
    if m['m00'] == 0: return (0.0, 1.0)
    mu20, mu02, mu11 = m['mu20'], m['mu02'], m['mu11']
    cov = np.array([[mu20, mu11],[mu11, mu02]], dtype=np.float64)
    w,v = np.linalg.eigh(cov)
    vmaj = v[:,1]
    n = np.hypot(vmaj[0], vmaj[1]) + 1e-9
    vx, vy = vmaj[0]/n, vmaj[1]/n
    return (vy, vx)  # (dy, dx)

def axis_phi_deg_from_unit(dy, dx):
    return math.degrees(math.atan2(dy, dx))  # y-down image

def connected_components(mask01):
    num, labels, stats, cents = cv2.connectedComponentsWithStats((mask01>0).astype(np.uint8), connectivity=8)
    return num, labels, stats, cents

def centroid_from_mask(mask01):
    m = cv2.moments((mask01*255).astype(np.uint8), binaryImage=True)
    if m['m00'] == 0: return None
    return (m['m01']/m['m00'], m['m10']/m['m00'])  # (cy,cx)

# ---------- D) Visualization helpers ----------
def draw_mask_contours(canvas_bgr, mask01, color, thickness=1):
    cnts, _ = cv2.findContours((mask01*255).astype(np.uint8), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    cv2.drawContours(canvas_bgr, cnts, -1, color, thickness)

def colorize_labels(labels):
    h, w = labels.shape[:2]
    n = int(labels.max()) + 1
    palette = np.zeros((max(n,1), 3), dtype=np.uint8)
    for lab in range(1, n):
        hue = int((lab * 37) % 180)  # OpenCV HSV hue 0..179
        palette[lab] = cv2.cvtColor(np.uint8([[[hue, 200, 255]]]), cv2.COLOR_HSV2BGR)[0, 0]
    return palette[labels]  # BGR uint8

# ---------- E) Listing & alignment ----------
def natural_key(s: str):
    return [int(t) if t.isdigit() else t.lower() for t in re.findall(r'\d+|\D+', s)]

def _list_images_sorted(dir_path, exts=(".png", ".jpg", ".jpeg", ".bmp")):
    if not os.path.isdir(dir_path): raise FileNotFoundError(f"Directory not found: {dir_path}")
    files = [os.path.join(dir_path, f) for f in os.listdir(dir_path) if os.path.splitext(f)[1].lower() in exts]
    files.sort(key=lambda p: natural_key(os.path.basename(p)))
    return files

def align_triplet(vid_dir: str, abd_dir: str, fb_dir: str):
    vid_list = _list_images_sorted(vid_dir)
    abd_list = _list_images_sorted(abd_dir)
    fb_list  = _list_images_sorted(fb_dir)
    n = min(len(vid_list), len(abd_list), len(fb_list))
    if n == 0:
        raise FileNotFoundError(
            "No images to align.\n"
            f"  input_vid: {len(vid_list)} in {vid_dir}\n"
            f"  abdomen_mask: {len(abd_list)} in {abd_dir}\n"
            f"  front_body_mask: {len(fb_list)} in {fb_dir}"
        )
    if len({len(vid_list), len(abd_list), len(fb_list)}) != 1:
        print(f"[warn] counts differ (vid, abd, fb) = {(len(vid_list), len(abd_list), len(fb_list))}. Using first {n} frames of each.")
    vid_list, abd_list, fb_list = vid_list[:n], abd_list[:n], fb_list[:n]
    base_list = [os.path.splitext(os.path.basename(p))[0] for p in vid_list]
    return vid_list, abd_list, fb_list, base_list

# ---------- F) Cone brightness ----------
def cone_brightness_score(gray, center_yx, axis_phi_deg, half_deg, length_px, rays):
    """
    Integrate grayscale intensity within a cone fan along an axis.
    Returns (mean_intensity, rays_lines) for debug drawing.
    """
    H, W = gray.shape
    cy, cx = center_yx
    total = 0.0; count = 0; rays_lines = []
    for a in np.linspace(-half_deg, +half_deg, rays):
        ang = math.radians(axis_phi_deg + a)
        ux, uy = math.cos(ang), math.sin(ang)  # y-down
        seg = []
        for r in range(1, length_px+1):
            y = int(round(cy + uy*r)); x = int(round(cx + ux*r))
            if 0 <= y < H and 0 <= x < W:
                seg.append((x, y))
                total += float(gray[y, x]); count += 1
            else:
                break
        if seg: rays_lines.append(seg)
    mean_intensity = total / (count + 1e-6)
    return mean_intensity, rays_lines

# ---------- G) Detector main ----------
def process_video_root(video_root, P, debug=False, debug_frame=30):
    H=W=850
    h_small, w_small = H//P["scale_factor"], W//P["scale_factor"]

    vid_dir = os.path.join(video_root, "input_vid")
    abd_dir = os.path.join(video_root, "abdomen_mask")
    fb_dir  = os.path.join(video_root, "front_body_mask")
    out_pt  = os.path.join(video_root, "simple_troph_point_heatmap")
    out_ln  = os.path.join(video_root, "simple_participant_line_heatmap")
    ensure_dirs(out_pt, out_ln)

    vid_list, abd_list, fb_list, base_list = align_triplet(vid_dir, abd_dir, fb_dir)
    n = len(base_list)

    dbg_dir = os.path.join(video_root, "_debug_troph")
    if debug: ensure_dirs(dbg_dir)

    next_id = 1
    prev_regions = {}
    tracks = {}

    for t in range(n):
        comp = read_rgb(vid_list[t])
        red  = comp[...,0]
        abd  = read_bin(abd_list[t])
        fb_g = read_gray(fb_list[t])

        # 01 inputs overlay
        if debug and t==debug_frame:
            vis = to_bgr(comp)
            draw_mask_contours(vis, abd, (0,255,0), 1)
            if fb_g.std() < 1e-6:
                fb_bin = (fb_g > int(fb_g.mean())).astype(np.uint8)
            else:
                _, fb_bin8 = cv2.threshold(fb_g, 0, 255, cv2.THRESH_BINARY+cv2.THRESH_OTSU)
                fb_bin = (fb_bin8 > 0).astype(np.uint8)
            draw_mask_contours(vis, fb_bin, (255,0,0), 1)
            cv2.imwrite(os.path.join(dbg_dir, "01_inputs.png"), vis)

        # labels
        num, labels, stats, cents = connected_components(abd)
        cur = []

        if debug and t == debug_frame:
            colored_bgr = colorize_labels(labels)
            cv2.imwrite(os.path.join(dbg_dir, "02_labels.png"), colored_bgr)

        for lab in range(1, num):
            area = int(stats[lab, cv2.CC_STAT_AREA])
            if area < 10: continue
            mask_l = (labels == lab).astype(np.uint8)
            cy, cx = float(cents[lab][1]), float(cents[lab][0])
            cur.append(dict(mask=mask_l, centroid=(cy, cx)))

        # assign to prev by IoU
        assignments, used = {}, set()
        for i, R in enumerate(cur):
            best = (0.0, None)
            for tid, pm in prev_regions.items():
                if tid in used: continue
                s = iou(R["mask"], pm)
                if s > best[0]: best = (s, tid)
            assignments[i] = best[1] if best[0] >= P["iou_match_threshold"] else None

        # update tracks with IoU + red proportion
        for i, R in enumerate(cur):
            tid = assignments[i]
            if tid is None:
                tid = next_id; next_id += 1
                tracks[tid] = dict(iou_hist=[], red_prop_hist=[])
            if tid in prev_regions:
                tracks[tid]["iou_hist"].append(iou(R["mask"], prev_regions[tid]))
            m = R["mask"].astype(bool)
            prop = float((red[m] >= P["brightnessThreshold"]).mean()) if m.sum()>0 else 0.0
            tracks[tid]["red_prop_hist"].append(prop)
            R["track_id"] = tid

        prev_regions = {R["track_id"]: R["mask"] for R in cur}

        # candidate regions
        candidates, cand_dbg = [], []
        for R in cur:
            tid = R["track_id"]
            iou_hist = tracks[tid]["iou_hist"]
            rp_hist  = tracks[tid]["red_prop_hist"]
            stationary = (len(iou_hist) >= max(1, P["min_duration"]-1) and
                          np.mean(iou_hist[-(P["min_duration"]-1):] or [0.0]) >= (1.0 - P["iou_stationary_tol"]))
            changing = False
            curm = prevm = None
            if len(rp_hist) >= P["smoothingWindow"] + 1:
                curm  = float(np.mean(rp_hist[-P["smoothingWindow"]:]))
                prevm = float(np.mean(rp_hist[-P["smoothingWindow"]-1:-1]))
                changing = abs(curm - prevm) >= P["brightPropDelta"]
            if stationary and changing:
                candidates.append(R)
            cand_dbg.append(dict(
                frame=t, tid=tid,
                iou_mean=float(np.mean(iou_hist[-(P["min_duration"]-1):] or [0.0])),
                rp_cur=curm, rp_prev=prevm,
                rp_delta=(None if curm is None or prevm is None else curm-prevm),
                stationary=bool(stationary), changing=bool(changing)
            ))

        if debug and t==debug_frame:
            vis = to_bgr(comp)
            for R in candidates:
                draw_mask_contours(vis, R["mask"], (0,255,255), 2)
                cy,cx = map(int, R["centroid"])
                cv2.putText(vis, f"id{R['track_id']}", (cx+5, cy-5),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,255,255), 1, cv2.LINE_AA)
            cv2.imwrite(os.path.join(dbg_dir, "03_candidates.png"), vis)
            with open(os.path.join(dbg_dir, "debug_log.txt"), "w") as f:
                for row in cand_dbg: f.write(json.dumps(row)+"\n")

        # head points via cone brightness on grayscale fb_g
        head_points, per_tid_circle = [], {}
        if debug and t==debug_frame:
            vis_axes = to_bgr(comp)

        for R in candidates:
            dy_u, dx_u = region_axis_unit(R["mask"])
            phi = axis_phi_deg_from_unit(dy_u, dx_u)
            cy, cx = R["centroid"]

            Fscore, rays_f = cone_brightness_score(fb_g, (cy, cx), phi,
                                                   P["coneAngle_deg"], P["coneLength"], P["rays_per_cone"])
            Bscore, rays_b = cone_brightness_score(fb_g, (cy, cx), phi+180.0,
                                                   P["coneAngle_deg"], P["coneLength"], P["rays_per_cone"])

            ratio = (Fscore - Bscore) / (Fscore + Bscore + 1e-6)
            sign = +1.0 if ratio >= 0.0 else -1.0

            hy = cy + sign * dy_u * P["abdomenToHeadLength"]
            hx = cx + sign * dx_u * P["abdomenToHeadLength"]
            hy, hx = clamp(hy, hx, H, W)
            head_points.append((hy, hx, R["track_id"]))

            if debug and t==debug_frame:
                p1 = (int(cx - dx_u*35), int(cy - dy_u*35))
                p2 = (int(cx + dx_u*35), int(cy + dy_u*35))
                cv2.arrowedLine(vis_axes, p1, p2, (255,255,255), 1, tipLength=0.2)
                for seg in rays_f:
                    for i in range(1,len(seg)): cv2.line(vis_axes, seg[i-1], seg[i], (0,0,255), 1)
                for seg in rays_b:
                    for i in range(1,len(seg)): cv2.line(vis_axes, seg[i-1], seg[i], (255,255,0), 1)
                cv2.putText(vis_axes, f"F:{Fscore:.1f} B:{Bscore:.1f} r:{ratio:.3f}",
                            (int(cx)+6, int(cy)+14), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,200,255), 1, cv2.LINE_AA)

        if debug and t==debug_frame:
            cv2.imwrite(os.path.join(dbg_dir, "04_axes_cones.png"), vis_axes)

        # --- outputs: draw directly on 170×170 canvases ---
        small_point = np.zeros((h_small, w_small), np.uint8)
        small_line  = np.zeros((h_small, w_small), np.uint8)

        if debug and t==debug_frame:
            vis_final = to_bgr(comp).copy()

        intersections = np.zeros((H, W), np.uint8)
        if head_points:
            head_sum = np.zeros((H, W), np.uint16)
            for hy, hx, tid in head_points:
                m = np.zeros((H, W), np.uint8)
                cv2.circle(m, (hx, hy), P["headRadius"], 1, thickness=-1)
                per_tid_circle[tid] = m
                head_sum += m.astype(np.uint16)
            intersections = (head_sum >= P["min_overlap_pairs"]).astype(np.uint8)

        # 05: headpoint circles & intersections
        if debug and t==debug_frame:
            vis_hp = to_bgr(comp).copy()
            for tid, m in per_tid_circle.items():
                cnts, _ = cv2.findContours((m*255), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
                cv2.drawContours(vis_hp, cnts, -1, (0,255,255), 1)
            if intersections.any():
                cnts, _ = cv2.findContours((intersections*255), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
                cv2.drawContours(vis_hp, cnts, -1, (255,0,255), 2)
                num_cc, labs = cv2.connectedComponents(intersections, connectivity=8)
                for lab in range(1, num_cc):
                    c = centroid_from_mask((labs==lab).astype(np.uint8))
                    if c is not None:
                        cy, cx = int(round(c[0])), int(round(c[1]))
                        cv2.circle(vis_hp, (cx, cy), 3, (255,0,255), -1)
            cv2.imwrite(os.path.join(dbg_dir, "05_headpoints_circles.png"), vis_hp)

        if intersections.any():
            num_cc, labs = cv2.connectedComponents(intersections, connectivity=8)
            for lab in range(1, num_cc):
                comp_mask = (labs==lab).astype(np.uint8)
                c = centroid_from_mask(comp_mask)
                if c is None: continue
                cy, cx = clamp(c[0], c[1], H, W)
                sy, sx = int(cy * h_small / H), int(cx * w_small / W)
                sy = min(max(sy, 0), h_small-1)
                sx = min(max(sx, 0), w_small-1)
                small_point[sy, sx] = 255

                tids = [tid for tid,m in per_tid_circle.items() if (comp_mask & (m>0)).any()]
                for tid in tids:
                    amask = prev_regions.get(tid, None)
                    if amask is None: continue
                    by, bx = find_nearest_boundary(amask, cy, cx)
                    sby, sbx = int(by * h_small / H), int(bx * w_small / W)
                    sby = min(max(sby, 0), h_small-1)
                    sbx = min(max(sbx, 0), w_small-1)
                    #cv2.line(small_line, (sx, sy), (sbx, sby), 255, 1)
                    small_line[sbx, sby] = 255

                    if debug and t==debug_frame:
                        cv2.circle(vis_final, (cx, cy), 3, (0,255,0), -1)
                        cv2.line(vis_final, (cx, cy), (bx, by), (0,255,0), 2)

        if debug and t==debug_frame:
            cv2.imwrite(os.path.join(dbg_dir, "06_outputs_overlay.png"), vis_final)

        # save SMALL images directly (no resizing step)
        base = base_list[t] + ".png"
        iio.imwrite(os.path.join(out_pt, base), small_point)
        iio.imwrite(os.path.join(out_ln, base), small_line)

        if (t+1) % 25 == 0 or t == n-1:
            print(f"[{t+1}/{n}]")

    print("Done.")


## 2B: Staging

In [5]:
# === Cell 2B — Staging (TAR-only, export *_simp.tar; optional raw copy) ===
# Assumptions:
#   • You provide a pre-made TAR on Drive: <stub>.tar (Windows "Send to → Compressed (zipped) → .tar").
#   • The TAR has a single wrapper folder containing: input_vid/, abdomen_mask/, front_body_mask/
#   • We NEVER build input archives in Colab; we only consume <stub>.tar.
# Behavior:
#   • Process locally for speed.
#   • DEFAULT: package entire local result as <stub>_simp.tar and upload it next to <stub>.tar.
#   • OPTIONAL: also copy raw output folders back to the Drive stub folder when COPY_RAW_OUTPUTS=True in Config.
#
# Required upstream:
#   • process_video_root(...) defined in Cell 2A.

import os
import shutil
import subprocess
import time

# Constants
LOCAL_STAGE = "/content/_localstage"
INPUT_DIRS  = ("input_vid", "abdomen_mask", "front_body_mask")
# Output subfolders our detector produces (extend if you add more)
OUTPUT_DIRS = ("simple_troph_point_heatmap", "simple_participant_line_heatmap", "_debug_troph")

# --- small helpers ---
def _ensure_dir(p: str):
    os.makedirs(p, exist_ok=True)

def _run(cmd, cwd=None, label=None, tolerate_patterns=None):
    """Run a command; on failure, show combined stdout/stderr (tail)."""
    import shlex
    p = subprocess.run(
        cmd,
        cwd=cwd,
        stdout=subprocess.PIPE,
        stderr=subprocess.STDOUT,
        text=True,
    )
    out = p.stdout or ""
    if p.returncode != 0:
        if tolerate_patterns and any(pat in out for pat in tolerate_patterns):
            print(f"[warn] tolerated nonzero exit for {label or cmd[0]} (rc={p.returncode})")
            print(out.rstrip()[-800:])
            return out
        tail = out[-2000:]
        raise RuntimeError(
            f"Command failed (rc={p.returncode}) [{label or cmd[0]}]\n"
            f"$ {shlex.join(cmd)}\n"
            f"--- stdout/stderr (tail) ---\n{tail}"
        )
    return out

def _tar_extract(tar_path: str, dest_dir: str):
    _ensure_dir(dest_dir)
    _run(["tar", "-xf", tar_path, "-C", dest_dir], label="tar-extract")

def _tar_create_dir(parent_dir: str, folder_name: str, out_tar_path: str):
    """Create a tar that contains `folder_name/` as the top-level entry."""
    _ensure_dir(os.path.dirname(out_tar_path))
    _run(["tar", "-cf", out_tar_path, folder_name], cwd=parent_dir, label="tar-create")

def _copy_dir_subset(src_root: str, dst_root: str, exclude_names):
    """Copy everything from src_root to dst_root except names in exclude_names."""
    _ensure_dir(dst_root)
    exclude = set(exclude_names)
    for name in os.listdir(src_root):
        if name in exclude:
            continue
        sp = os.path.join(src_root, name)
        dp = os.path.join(dst_root, name)
        if os.path.isdir(sp):
            if os.path.exists(dp):
                shutil.rmtree(dp, ignore_errors=True)
            shutil.copytree(sp, dp)
        else:
            shutil.copy2(sp, dp)

def _canonicalize_inputs_root(root: str):
    """
    After extraction, ensure `root` has input_vid/, abdomen_mask/, front_body_mask/ directly under it.
    If there's exactly one wrapper directory, move *all* of its contents up (not just the inputs).
    """
    # Already canonical?
    if all(os.path.isdir(os.path.join(root, d)) for d in INPUT_DIRS):
        return

    # Find single wrapper
    entries = [e for e in os.listdir(root) if os.path.isdir(os.path.join(root, e))]
    if len(entries) != 1:
        raise FileNotFoundError(f"Expected a single wrapper folder inside {root}, found: {entries}")

    wrapper = os.path.join(root, entries[0])
    # Validate inputs exist inside wrapper (to avoid promoting wrong folder)
    if not all(os.path.isdir(os.path.join(wrapper, d)) for d in INPUT_DIRS):
        raise FileNotFoundError(f"Wrapper {wrapper} does not contain required {INPUT_DIRS}")

    # Move EVERYTHING (files + folders) from wrapper/* up to root/*
    for name in os.listdir(wrapper):
        src = os.path.join(wrapper, name)
        dst = os.path.join(root, name)
        # No conflicts expected (root only holds the wrapper), but be safe:
        if os.path.exists(dst):
            # If conflict, remove destination then move
            if os.path.isdir(dst):
                shutil.rmtree(dst, ignore_errors=True)
            else:
                os.remove(dst)
        shutil.move(src, dst)

    # Remove now-empty wrapper
    shutil.rmtree(wrapper, ignore_errors=True)


def _print_inputs_sanity(root: str, max_show=2):
    """Optional: quick counts and a couple of sample names per input dir."""
    import glob
    for d in INPUT_DIRS:
        folder = os.path.join(root, d)
        files = sorted(glob.glob(os.path.join(folder, "*.png")) + glob.glob(os.path.join(folder, "*.PNG")))
        n = len(files)
        head = [os.path.basename(x) for x in files[:max_show]]
        tail = [os.path.basename(x) for x in files[-max_show:]] if n > max_show else []
        print(f"[sanity] {d}: {n} PNGs", f"head={head}" if head else "", f"tail={tail}" if tail else "")

def _count_min_frames(root: str) -> int:
    """Minimum frame count across the three input dirs."""
    import glob
    def _cnt(sub):
        p = os.path.join(root, sub)
        return len(glob.glob(os.path.join(p, "*.png")) + glob.glob(os.path.join(p, "*.PNG")))
    return min(_cnt("input_vid"), _cnt("abdomen_mask"), _cnt("front_body_mask"))

# --- main wrapper (TAR-only; exports *_simp.tar; optional raw copy) ---
def process_video_root_localfirst(
    drive_video_root: str,
    P: dict,
    debug: bool = False,
    debug_frame: int | None = None,
    keep_local: bool = False,
    copy_raw_outputs: bool = None,   # None → read from global COPY_RAW_OUTPUTS (default False)
):
    """
    Consume pre-supplied <stub>.tar by appending '.tar' to `drive_video_root`.
    Process locally, then:
      • ALWAYS create <stub>_simp.tar from the local result and place it next to <stub>.tar on Drive.
      • OPTIONALLY also copy raw outputs back to the Drive stub folder when copy_raw_outputs=True.
    """
    # resolve optional flag from Config if not provided
    if copy_raw_outputs is None:
        copy_raw_outputs = bool(globals().get("COPY_RAW_OUTPUTS", False))

    base      = os.path.basename(os.path.normpath(drive_video_root))     # e.g., "tmpvidroot3"
    drive_tar = drive_video_root.rstrip("/\\") + ".tar"                  # …/tmpvidroot3.tar
    if not os.path.exists(drive_tar):
        raise FileNotFoundError(f"Archive not found: {drive_tar}")

    local_root      = os.path.join(LOCAL_STAGE, base)                     # /content/_localstage/tmpvidroot3
    temp_tar_local  = os.path.join(LOCAL_STAGE, f"{base}__inputs.tar")
    result_tar_local = os.path.join(LOCAL_STAGE, f"{base}_simp.tar")      # local packaged result
    out_tar_drive    = os.path.join(os.path.dirname(drive_tar), f"{base}_simp.tar")

    # fresh local workspace
    if os.path.isdir(local_root):
        shutil.rmtree(local_root, ignore_errors=True)
    _ensure_dir(LOCAL_STAGE)
    _ensure_dir(local_root)

    # stage-in: copy one big file, extract, canonicalize
    t0 = time.time()
    print(f"[stage] Using TAR -> local: {drive_tar} -> {local_root}")
    shutil.copy2(drive_tar, temp_tar_local)
    _tar_extract(temp_tar_local, local_root)
    os.remove(temp_tar_local)
    _canonicalize_inputs_root(local_root)
    _print_inputs_sanity(local_root, max_show=2)
    print(f"[stage] Stage-in done in {time.time()-t0:.1f}s")

    # clamp debug_frame so we always hit a valid frame if debug=True
    if debug and debug_frame is not None:
        n = _count_min_frames(local_root)
        if n <= 0:
            df = 0
        elif debug_frame < 0:
            df = n - 1
        elif debug_frame >= n:
            df = n - 1
        else:
            df = debug_frame
        print(f"[stage] Using debug_frame={df} (clamped)")
    else:
        df = debug_frame
        print("[stage] Debug disabled")

    # run locally
    t1 = time.time()
    print("[run] process_video_root(...) on local copy")
    result = process_video_root(local_root, P, debug=debug, debug_frame=df)
    print(f"[run] done in {time.time()-t1:.1f}s")

    # PACKAGE RESULT: tar the entire local video root (inputs + outputs)
    t2 = time.time()
    print(f"[export] Packaging result to: {result_tar_local}")
    _tar_create_dir(parent_dir=os.path.dirname(local_root), folder_name=os.path.basename(local_root), out_tar_path=result_tar_local)
    print(f"[export] Uploading packaged tar to Drive: {out_tar_drive}")
    shutil.copy2(result_tar_local, out_tar_drive)
    os.remove(result_tar_local)
    print(f"[export] Packaging + upload done in {time.time()-t2:.1f}s")

    # OPTIONAL: also copy raw outputs back to Drive folder
    if copy_raw_outputs:
        t3 = time.time()
        out_dest = drive_video_root  # ensure folder exists
        _ensure_dir(out_dest)
        print("[stage] Copying raw outputs back to Drive stub folder (inputs skipped)")
        _copy_dir_subset(local_root, out_dest, exclude_names=INPUT_DIRS)
        print(f"[stage] Copy-back done in {time.time()-t3:.1f}s")
        # confirm debug folder presence
        if debug:
            dbg = os.path.join(out_dest, "_debug_troph")
            if os.path.isdir(dbg):
                import glob
                n_dbg = len(glob.glob(os.path.join(dbg, "*.png")))
                print(f"[stage] Debug folder present: {dbg}  ({n_dbg} PNGs)")
            else:
                print("[stage] Note: no _debug_troph folder on Drive (images only written when t == debug_frame).")
    else:
        print("[stage] Skipping raw output copy (COPY_RAW_OUTPUTS=False).")

    # cleanup local
    if keep_local:
        print(f"[stage] Keeping local: {local_root}")
    else:
        shutil.rmtree(local_root, ignore_errors=True)
        print("[stage] Local staging cleaned")

    return result


In [6]:
# === Cell 2C — Fake data suite (definitions only; matches original behavior) ===
# Folders expected by the detector:
#   input_vid/ (RGB), abdomen_mask/ (binary 0/255), front_body_mask/ (binary 0/255)
# Default local root for fake data:
FAKE_VIDEO_ROOT = "/content/fake_video_root"

import os, math, shutil
import numpy as np, cv2
from imageio import v3 as iio

# -- helpers -------------------------------------------------
def _ellipse_mask(H: int, W: int, cx: float, cy: float, a: float, b: float, angle_deg: float) -> np.ndarray:
    """Filled ellipse mask (uint8 0/255). Note: cv2.ellipse uses (x, y) order for center and axes."""
    m = np.zeros((H, W), np.uint8)
    cv2.ellipse(m, (int(cx), int(cy)), (int(a), int(b)), float(angle_deg), 0, 360, 255, -1)
    return m

def _triangle_wedge(H: int, W: int, cx: float, cy: float, axis_deg: float, dir_sign: int, length: int = 60, base: int = 30) -> np.ndarray:
    """Isosceles triangular wedge pointing along +/− axis; returns uint8 mask 0/255."""
    # axis_deg: 0° points right; positive angles rotate counterclockwise in image coords
    ux, uy = math.cos(math.radians(axis_deg)), -math.sin(math.radians(axis_deg))
    px, py = cx + dir_sign * ux * length, cy + dir_sign * uy * length
    vx, vy = -uy, ux
    b = base / 2.0
    blx, bly = cx - vx * b, cy - vy * b
    brx, bry = cx + vx * b, cy + vy * b
    poly = np.array([[blx, bly], [brx, bry], [px, py]], dtype=np.float32)
    poly[:, 0] = np.clip(np.round(poly[:, 0]), 0, W - 1)
    poly[:, 1] = np.clip(np.round(poly[:, 1]), 0, H - 1)
    m = np.zeros((H, W), np.uint8)
    cv2.fillConvexPoly(m, poly.astype(np.int32), 255)
    return m

def _reset_dir(d: str) -> None:
    """Delete and recreate a directory."""
    if os.path.isdir(d):
        shutil.rmtree(d)
    os.makedirs(d, exist_ok=True)

# -- main generator ------------------------------------------
def generate_fake_troph_data(
    video_root: str,
    n_frames: int = 60,
    seed: int = 0,
    prefix: str = "frame_",
    start_index: int = 1,
    pad: int = 5,
) -> None:
    """
    Synthesize a video_root with the exact structure your detector expects:
      • input_vid/         (RGB 850×850)     — RED channel modulated over time
      • abdomen_mask/      (binary 0/255)    — multiple ellipses (some moving)
      • front_body_mask/   (binary 0/255)    — triangular 'forward' wedges
    Prints periodic progress identical to your previous behavior.
    """
    rng = np.random.default_rng(seed)
    H, W = 850, 850

    out_vid = os.path.join(video_root, "input_vid")
    out_abd = os.path.join(video_root, "abdomen_mask")
    out_fb  = os.path.join(video_root, "front_body_mask")

    # clean and recreate input folders
    _reset_dir(out_vid)
    _reset_dir(out_abd)
    _reset_dir(out_fb)

    # scene: 5 abdomens (2 stationary feeding, 1 stationary quiet, 2 moving)
    A = [
        dict(cx=375.0, cy=420.0, a=28, b=18, axis_deg=0.0,   dir_sign=+1, kind="stationary_feed"),
        dict(cx=435.0, cy=420.0, a=28, b=18, axis_deg=180.0, dir_sign=+1, kind="stationary_feed"),
        dict(cx=200.0, cy=220.0, a=26, b=16, axis_deg=45.0,  dir_sign=+1, kind="stationary_quiet"),
        dict(cx=620.0, cy=320.0, a=26, b=16, axis_deg=75.0,  dir_sign=+1, kind="moving"),
        dict(cx=180.0, cy=600.0, a=30, b=20, axis_deg=135.0, dir_sign=+1, kind="moving"),
    ]
    vel = {3: (+0.8, +0.5), 4: (+0.6, -0.7)}  # indices into A

    # static noise map to create consistent red boost regions over time
    noise = rng.random((H, W))  # float64 0..1 is fine

    # step schedule ensures detectable Δ in red proportion
    steps = [0.10, 0.32, 0.55, 0.30]
    step_len = max(5, n_frames // len(steps))

    for t in range(n_frames):
        comp_rgb = np.full((H, W, 3), 235, np.uint8)  # light background
        abd_mask_all = np.zeros((H, W), np.uint8)
        fb_mask_all  = np.zeros((H, W), np.uint8)
        red_hi_all   = np.zeros((H, W), dtype=bool)

        k = min(t // step_len, len(steps) - 1)
        p_feed, p_quiet = steps[k], 0.10

        for i, a in enumerate(A):
            if a["kind"] == "moving":
                dx, dy = vel[i]
                a["cx"] += dx; a["cy"] += dy
                # bounce within margins
                if not (80 < a["cx"] < W - 80): vel[i] = (-dx, dy); a["cx"] = np.clip(a["cx"], 80, W - 80)
                if not (80 < a["cy"] < H - 80): vel[i] = (dx, -dy); a["cy"] = np.clip(a["cy"], 80, H - 80)

            cx, cy = a["cx"], a["cy"]
            m_abd = _ellipse_mask(H, W, cx, cy, a["a"], a["b"], a["axis_deg"])
            m_fb  = _triangle_wedge(H, W, cx, cy, a["axis_deg"], a["dir_sign"], length=60, base=30)

            abd_mask_all |= (m_abd > 0).astype(np.uint8)
            fb_mask_all  |= (m_fb  > 0).astype(np.uint8)

            p = p_feed if a["kind"] == "stationary_feed" else p_quiet
            red_hi_all |= (noise < p) & (m_abd > 0)

        # compose RGB: dark body; base red inside abdomen; extra bright red on "feeding" pixels
        comp_rgb[abd_mask_all > 0] = (60, 60, 60)   # dark body
        comp_rgb[..., 0][abd_mask_all > 0] = 120    # base red
        comp_rgb[..., 0][red_hi_all] = 220          # boosted red (feeding)

        base = f"{prefix}{start_index + t:0{pad}d}.png"
        iio.imwrite(os.path.join(out_vid, base), comp_rgb)                          # RGB
        iio.imwrite(os.path.join(out_abd, base), (abd_mask_all * 255).astype(np.uint8))  # 0/255
        iio.imwrite(os.path.join(out_fb,  base), (fb_mask_all  * 255).astype(np.uint8))  # 0/255

        if (t + 1) % 20 == 0 or t == n_frames - 1:
            print(f"[fake {t+1}/{n_frames}] wrote {base}")

    print(
        "Fake data ready:",
        f"\n  input_vid:        {len(os.listdir(out_vid))} PNGs",
        f"\n  abdomen_mask:     {len(os.listdir(out_abd))} PNGs",
        f"\n  front_body_mask:  {len(os.listdir(out_fb))} PNGs",
    )

# -- convenience runner --------------------------------------
def run_fake_local(
    P: dict,
    video_root: str = FAKE_VIDEO_ROOT,
    debug: bool = True,
    debug_frame: int = 30,
    n_frames: int = 60,
    rebuild: bool = False,
):
    """
    Generate fake data under `video_root` (local) and invoke the detector:
        process_video_root(video_root, P, debug=..., debug_frame=...)
    Set rebuild=True to wipe any existing fake input folders before generating.
    """
    if rebuild and os.path.isdir(video_root):
        shutil.rmtree(video_root, ignore_errors=True)
    os.makedirs(video_root, exist_ok=True)

    # (Re)generate inputs if missing or on rebuild
    if rebuild or not all(os.path.isdir(os.path.join(video_root, d)) for d in ("input_vid","abdomen_mask","front_body_mask")):
        generate_fake_troph_data(video_root, n_frames=n_frames, seed=0)

    # rely on detector’s process_video_root (defined in your detector cell)
    return process_video_root(video_root, P, debug=debug, debug_frame=debug_frame)


## Cell 3 Orchestrator

In [7]:
# === Cell 3 — Orchestrator (TAR-only; exports *_simp.tar; optional raw copy; uses FIELD_PARAMS) ===
# Expects from Config (Cell 1):
#   RUN_FAKE, RUN_TEST, RUN_BATCH : bool
#   FIELD_PARAMS                  : dict   <-- used for all runs
#   TEST_VIDEO_ROOT               : str    (Drive stub folder path; TAR is stub + ".tar")
#   BATCH_ROOT                    : str    (Drive parent folder for stubs)
#   BATCH_DIR_LIST                : str    (path to .txt list of stub names or absolute paths)
#   DEBUG                         : bool
#   DEBUG_FRAME                   : int
# Optional:
#   KEEP_LOCAL                    : bool   (keep staged local copy after run)
#   COPY_RAW_OUTPUTS              : bool   (default False; also copy raw outputs back to Drive)

import os
import time

REQUIRED_INPUTS = ("input_vid", "abdomen_mask", "front_body_mask")

def _iter_batch_roots(batch_root: str, list_file: str):
    """Yield absolute stub folder paths from a list file.
    - Lines can be relative names (joined to batch_root) or absolute paths.
    - We *do not* add '.tar' here; the staging wrapper does that.
    - Ignores empty lines and lines starting with '#'.
    """
    if not os.path.isfile(list_file):
        print(f"[batch] list file not found: {list_file}")
        return
    with open(list_file, "r", encoding="utf-8") as f:
        for line in f:
            name = line.strip()
            if not name or name.startswith("#"):
                continue
            yield name if os.path.isabs(name) else os.path.join(batch_root, name)

def _need(varname: str):
    if varname not in globals():
        raise RuntimeError(f"Config variable `{varname}` is not defined. Please set it in the Config cell.")
    return globals()[varname]

def run_all_from_config():
    """Run fake/test/batch according to Config flags. Returns a summary dict."""
    params           = _need("FIELD_PARAMS")
    run_fake         = bool(globals().get("RUN_FAKE", False))
    run_test         = bool(globals().get("RUN_TEST", False))
    run_batch        = bool(globals().get("RUN_BATCH", False))
    debug            = bool(globals().get("DEBUG", False))
    debug_frame      = int(globals().get("DEBUG_FRAME", 30))
    keep_local       = bool(globals().get("KEEP_LOCAL", False))
    copy_raw_outputs = bool(globals().get("COPY_RAW_OUTPUTS", False))

    summary = {"fake": None, "test": None, "batch": None}
    t0 = time.time()

    # --- FAKE (always local; no packaging requirement for Drive) ---
    if run_fake:
        print("[orchestrator] RUN_FAKE = True → run_fake_local()")
        try:
            run_fake_local(P=params, debug=debug, debug_frame=debug_frame)
            summary["fake"] = "ok"
        except Exception as e:
            print(f"[fake] ❌ {e}")
            summary["fake"] = f"error: {e}"

    # --- TEST (TAR-only; stub + '.tar'; exports *_simp.tar) ---
    if run_test:
        test_root = _need("TEST_VIDEO_ROOT")
        print(f"[orchestrator] RUN_TEST = True → {test_root} (.tar consumed; *_simp.tar produced)")
        try:
            process_video_root_localfirst(
                drive_video_root=test_root,
                P=params,
                debug=debug,
                debug_frame=debug_frame,
                keep_local=keep_local,
                copy_raw_outputs=copy_raw_outputs,
            )
            summary["test"] = "ok"
        except Exception as e:
            print(f"[test] ❌ {e}")
            summary["test"] = f"error: {e}"

    # --- BATCH (iterate stubs; TAR-only; exports *_simp.tar per stub) ---
    if run_batch:
        batch_root    = _need("BATCH_ROOT")
        batch_list_fn = _need("BATCH_DIR_LIST")
        print(f"[orchestrator] RUN_BATCH = True → list: {batch_list_fn}")
        ok = 0
        fail = 0
        failures = []
        for stub in _iter_batch_roots(batch_root, batch_list_fn):
            try:
                print(f"[batch] processing: {stub} (.tar consumed; *_simp.tar produced)")
                process_video_root_localfirst(
                    drive_video_root=stub,
                    P=params,
                    debug=debug,              # honor DEBUG for batch
                    debug_frame=debug_frame,
                    keep_local=False,
                    copy_raw_outputs=copy_raw_outputs,
                )
                ok += 1
            except Exception as e:
                fail += 1
                failures.append((stub, str(e)))
                print(f"[batch] ❌ {stub}: {e}")
        summary["batch"] = {"success": ok, "fail": fail, "failures": failures}

    print(f"[orchestrator] done in {time.time()-t0:.1f}s")
    return summary

# Tip:
# RESULT = run_all_from_config()
# RESULT


## Cell 4 Entry point

In [8]:
# === Cell 4 — Entry point ===
from pprint import pprint
import time

print("[entry] starting run_all_from_config() …")
t0 = time.time()
try:
    RESULT = run_all_from_config()
finally:
    print(f"[entry] finished in {time.time()-t0:.1f}s")

print("\n=== Summary ===")
pprint(RESULT)


[entry] starting run_all_from_config() …
[orchestrator] RUN_BATCH = True → list: /content/drive/MyDrive/MainConnection_VidRoots/batch_dir_list.txt
[batch] processing: /content/drive/MyDrive/MainConnection_VidRoots/tmpvidroot3 (.tar consumed; *_simp.tar produced)
[stage] Using TAR -> local: /content/drive/MyDrive/MainConnection_VidRoots/tmpvidroot3.tar -> /content/_localstage/tmpvidroot3
[sanity] input_vid: 132 PNGs head=['frame_000001.png', 'frame_000002.png'] tail=['frame_000131.png', 'frame_000132.png']
[sanity] abdomen_mask: 132 PNGs head=['frame_00001.png', 'frame_00002.png'] tail=['frame_00131.png', 'frame_00132.png']
[sanity] front_body_mask: 132 PNGs head=['frame_000386.png', 'frame_000387.png'] tail=['frame_000516.png', 'frame_000517.png']
[stage] Stage-in done in 0.4s
[stage] Using debug_frame=3 (clamped)
[run] process_video_root(...) on local copy
[25/132]
[50/132]
[75/132]
[100/132]
[125/132]
[132/132]
Done.
[run] done in 50.0s
[export] Packaging result to: /content/_localst

# Run up to this point

```
# This is formatted as code
```

